In [1]:
import tomllib
import requests
import os 

with open("config.toml", "rb") as f:
    cfg = tomllib.load(f)["binance"]

api_key = os.getenv("BINANCE_API_KEY")
headers = {
    "X-MBX-APIKEY": api_key
}
params  = {
    "symbol":   cfg["symbol"],
    "interval": cfg["interval"],
    "limit":    int(cfg["limit"]), 
}

url = cfg['url']
response = requests.get(url, headers=headers, params=params)

data = response.json()

open_time = data[0][0]
open_price = data[0][1]
high_price = data[0][2]
low_price = data[0][3]
close_price = data[0][4]
volume = data[0][5]

print(f"Open: {open_price}, High: {high_price}, Low: {low_price}, Close: {close_price}, Volume: {volume}")


Open: 107108.05000000, High: 107585.12000000, Low: 106866.63000000, Close: 107415.00000000, Volume: 966.52615000


In [1]:
import tomllib
import requests
import os
import time
import pandas as pd

In [2]:
with open("config.toml", "rb") as f:
    cfg = tomllib.load(f)["binance"]

api_key = os.getenv("BINANCE_API_KEY")
headers = {"X-MBX-APIKEY": api_key}

params = {
    "symbol":   cfg["symbol"],
    "interval": cfg["interval"],
    "limit":    int(cfg["limit"]), 
}
url = cfg['url']

In [4]:
df = pd.DataFrame(columns=["timestamp", "open", "high", "low", "close", "volume"])

try:
    while True:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code != 200:
            print("Error:", response.status_code, response.text)
            time.sleep(5)
            continue

        data = response.json()[0]  
        open_time = data[0]
        open_price = data[1]
        high_price = data[2]
        low_price = data[3]
        close_price = data[4]
        volume = data[5]

        if open_time not in df["timestamp"].values:
            new_row = {
                "timestamp": open_time,
                "open": float(open_price),
                "high": float(high_price),
                "low": float(low_price),
                "close": float(close_price),
                "volume": float(volume)
            }
            df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
            print("✅ New data collected.")
        else:
            print("🔁 No new data.")

        time.sleep(5)

except KeyboardInterrupt:
    print("\n⏹️ Data collection stopped by user.")

df.to_csv("ohlcv_data.csv", index=False)


/var/folders/8r/f598trz55xnb0kvpw1j3cglh0000gn/T/ipykernel_71081/3224718228.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)


✅ New data collected.
🔁 No new data.
✅ New data collected.
🔁 No new data.
🔁 No new data.
🔁 No new data.
🔁 No new data.
🔁 No new data.
🔁 No new data.
🔁 No new data.
🔁 No new data.
🔁 No new data.
🔁 No new data.
✅ New data collected.
🔁 No new data.
🔁 No new data.
🔁 No new data.
🔁 No new data.
🔁 No new data.
🔁 No new data.
🔁 No new data.
🔁 No new data.
🔁 No new data.
✅ New data collected.
🔁 No new data.

⏹️ Data collection stopped by user.


In [1]:
import psycopg2

# Replace these with your actual credentials
conn = psycopg2.connect(
    host="localhost",        # or your DB host/IP
    port="5432",             # default PostgreSQL port
    database="crypto",
    user="postgres",
    password="2002"
)

In [2]:
cursor = conn.cursor()

insert_query = """
INSERT INTO ohlcv (timestamp, open, high, low, close, volume)
VALUES (%s, %s, %s, %s, %s, %s);
"""
data = (
    1751130720000,
    107413.33,
    107413.33,
    107408.01,
    107408.01,
    0.84863
)

cursor.execute(insert_query, data)
conn.commit()

# Don't forget to close
cursor.close()
conn.close()


In [4]:
from app import config
from app.binance_client import BinanceClient

binance_client = BinanceClient(
    api_key=config.BINANCE_API_KEY,
    symbol=config.BINANCE_SYMBOL,
    interval=config.BINANCE_INTERVAL,
    limit=config.BINANCE_LIMIT,
    url=config.BINANCE_URL
).get_historical_data()


In [6]:
from binance import AsyncClient, BinanceSocketManager


ModuleNotFoundError: No module named 'binance'

In [7]:
!pip install binance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 850.2/850.2 kB 1.1 MB/s eta 0:00:00a 0:00:01


In [13]:
import asyncio
import json
import os
import tomllib
import websockets
from dotenv import load_dotenv
from app.database import insert_candle
from app import config
import nest_asyncio
nest_asyncio.apply()  # To allow nested event loops in Jupyter

load_dotenv()


symbol   = config.BINANCE_SYMBOL.lower()
interval = config.BINANCE_INTERVAL
stream_url = f"wss://stream.binance.com:9443/ws/{symbol}@kline_{interval}"

async def stream_data():
    async with websockets.connect(stream_url, ping_interval=20) as ws:
        async for message in ws:
            data = json.loads(message)
            kline = data["k"]
            if kline["x"]:  # closed candle only
                candle = {
                    "t": kline["t"],
                    "o": float(kline["o"]),
                    "h": float(kline["h"]),
                    "l": float(kline["l"]),
                    "c": float(kline["c"]),
                    "v": float(kline["v"])
                }
                await insert_candle(candle)
                print("Saved candle:", candle)

if __name__ == "__main__":
    asyncio.run(stream_data())


KeyboardInterrupt: 

In [17]:
data

NameError: name 'data' is not defined

<code object connect at 0x119a456d0, file "/opt/anaconda3/envs/studentgator/lib/python3.11/site-packages/asyncpg/connection.py", line 2074>